### You can change the pretrained model as you like (VGG19, RESNET50, EFFINETB0,....)

## First Methode 

In [ ]:
import os
import numpy as np
from PIL import Image
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import Model

In [ ]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
model = Model(inputs=base_model.input, outputs=base_model.output)

def extract_features(directory, label):
    features = []
    labels = []
    for file in os.listdir(directory):
        if file.endswith('.jpg'):
            img_path = os.path.join(directory, file)
            img = Image.open(img_path).convert('RGB').resize((224, 224))
            img_array = img_to_array(img)
            img_array = np.expand_dims(img_array, axis=0)
            img_array = preprocess_input(img_array)  # VGG-specific preprocessing

            feature = model.predict(img_array)
            features.append(feature.flatten())  # Flatten the conv output
            labels.append(label)
    return features, labels

benign_features, benign_labels = extract_features('C:/path', 0)
malignant_features, malignant_labels = extract_features('C:/path', 1)
# Combine and create dataset
X = np.array(benign_features + malignant_features)
y = np.array(benign_labels + malignant_labels)

In [ ]:
X

In [ ]:
y

## Second Methode

In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
import numpy as np
import os
from PIL import Image

# Load pre-trained VGG16 (exclude classifier)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
model = Model(inputs=base_model.input, outputs=base_model.output)

def extract_sequence_features(folder_path, label):
    features = []
    labels = []

    for img_file in os.listdir(folder_path):
        if img_file.endswith('.jpg'):
            path = os.path.join(folder_path, img_file)
            img = Image.open(path).convert('RGB').resize((224, 224))
            img_array = img_to_array(img)
            img_array = np.expand_dims(img_array, axis=0)
            img_array = preprocess_input(img_array)

            # Shape: (1, 7, 7, 512)
            feature = model.predict(img_array)
            feature_seq = feature.reshape(49, 512)  # Convert (7x7, 512) into 49 steps
            features.append(feature_seq)
            labels.append(label)

    return features, labels

In [ ]:
X_benign, y_benign = extract_sequence_features('C:/path', 0)
X_malignant, y_malignant = extract_sequence_features('C:/path', 1)

X1 = np.array(X_benign + X_malignant)  # shape: (samples, 49, 512)
y1 = np.array(y_benign + y_malignant)

In [ ]:
X1

In [ ]:
y1

#### Now you can use the arrays in any deep learnig model (CNN, LSTM, BILSTM, RNN, GRU,...) 